# Calculate Z-scores for players

This takes the list of players that we're trying to calculate scores for, and creates z-scores for players with an arbitrarily chosen minimum innings (for pitchers) or plate appearances (for hitters).

Notes for this year's sheet:

- Z-score avg/stdev calculated using minimum IP (70) or PA (35) stats for batting and pitching.
- Reliever value was artificially decremented as usual since they're normally over-inflated value-wise. Multiplied by (project innings) / (90th pctile of projected innings), capped at 1.0.

In [1]:
import pandas as pd
import numpy as np
import os
import sqlalchemy
import psycopg2
import seaborn as sb
import requests

# pandas display settings for ipynb
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)
pd.set_option('display.float_format', '{:20,.2f}'.format)
pd.set_option('display.max_colwidth', None)

## SqlAlchemy Connection Information

These are used to get and return a connection to the postgres DB so that we can query for player stats and write them back to the table.

In [2]:
# connection information for the database
POSTGRES_USER = os.environ.get("POSTGRES_USER")
POSTGRES_PASSWORD = os.environ.get("POSTGRES_PASSWORD")
POSTGRES_IP = "192.168.0.118"
POSTGRES_PORT = 5432
POSTGRES_DB = "postgres"


def get_sqlalchemy_engine():
    """
    Create and return a SQLAlchemy engine for inserting into postgres.
    """
    # ## Write Information Back to Database
    #
    return sqlalchemy.create_engine(
        "postgresql://{user}:{password}@{host}:{port}/{db}".format(
            user=POSTGRES_USER,
            password=POSTGRES_PASSWORD,
            host=POSTGRES_IP,
            port=POSTGRES_PORT,
            db=POSTGRES_DB,
        )
    )

In [3]:
# create a connection and read in pitchers data
engine = get_sqlalchemy_engine()
conn = engine.connect()

## Get Player Data and Create Calculated Columns

For each of the players, we calculate a z-score based on how they compare to the rest of the league. There are several columns that we want to do this for.

- For each z-score, there is an arbitrary minimum set to inflate the league average away from non-full-time players.
- After each is totaled, the z-scores are totaled and then z-scored again to make a single number. This is done so that we can compare pitchers and batters into the same chart.

In [4]:
# read in information from postgres
dfp = pd.read_sql(
    "select * from fantasy.pitchers_projections_depth_charts_ros where ip > 0.0", conn
)
dfb = pd.read_sql("select *from fantasy.batters_projections_depth_charts_ros", conn)
dfpa = pd.read_sql("select * from fantasy.pitchers_actuals", conn)
dfba = pd.read_sql("select * from fantasy.batters_actuals", conn)
dfbsa = pd.read_sql("select * from fantasy.batters_statcast", conn)
# dfpa.fillna(0, inplace=True)
# dfba.fillna(0, inplace=True)
print()

In [5]:
# create any calculated columns necessary
dfb['k_pct'] = (dfb['so'] / dfb['ab']).round(2)
dfb['rc'] = (dfb['ab'] * dfb['obp'] * dfb['slg']).round(2)
dfb['catg'] = "b"

dfba['k_pct'] = (dfb['so'] / dfb['ab']).round(2)
dfba['ab'] = (dfba['pa'] * dfba['bb_pct'].apply(lambda x: 1 - float(x.strip('%')) / 100)).astype(int)
dfba['rc'] = (dfba['ab'] * dfba['obp'] * dfba['slg']).round(2)
dfba['ops'] = dfba['slg'] + dfba['obp']
dfba['babip_diff'] = dfba['babip'] - dfba['avg']
dfba["catg"] = "b"

# quality starts metric from here:
# https://yourfantasyjoe.blogspot.com/2010/02/projecting-quality-starts.html
# xQS = GS * (.4650115 - (ERA * .0872381) + ((IP/GS) * .0746775))
dfp['qs'] = (dfp['gs'] * (0.4650115 - (dfp['era'] * 0.0872381) + ((dfp['ip']/dfp['gs']) * .0746775))).round(2)
dfp["catg"] = "p"
dfp.fillna(value={'qs': 0.0}, inplace=True)

dfpa["catg"] = "p"

print()

In [6]:
# thresholds for batters and pitchers
MINIMUM_INNINGS_PITCHED = 100
MINIMUM_INNINGS_PITCHED_ACTUALS = 4.0
MINIMUM_PLATE_APPEARANCES = 180
MINIMUM_PLATE_APPEARANCES_ACTUALS = 20
min_pa = dfb['pa'] > MINIMUM_PLATE_APPEARANCES
min_pa_act = dfba['pa'] > MINIMUM_PLATE_APPEARANCES_ACTUALS
min_ip = dfp['ip'] > MINIMUM_INNINGS_PITCHED
min_ip_act = dfpa['ip'] > MINIMUM_INNINGS_PITCHED_ACTUALS

# dict of columns that we want to calculate z-scores for
# +1 means more is better, -1 means lower is better
dfb_score_cols = {
    "pa": {"dir": 1, "weight": 1.3}, 
    "so": {"dir": -1, "weight": 1.0},
    "hr": {"dir": 1, "weight": 0.9},
    "rc": {"dir": 1, "weight": 1.4},
    "woba": {"dir": 1, "weight": 1.3},
    "slg": {"dir": 1, "weight": 1.0},
    "adp": {"dir": -1, "weight": 0.3}
}
dfp_score_cols = {
    "ip": {"dir": 1, "weight": 1.3},
    "era": {"dir": -1, "weight": 1.0},
    "hr": {"dir": -1, "weight": 0.9},
    "so": {"dir": 1, "weight": 1.0},
    "whip": {"dir": -1, "weight": 1.5},
    "k-9": {"dir": 1, "weight": 1.3},
    "qs": {"dir": 1, "weight": 1.0},
    "gs": {"dir": 1, "weight": 0.35}
}
dfba_score_cols = {
    "pa": {"dir": 1, "weight": 2.15},
    "k_pct": {"dir": -1, "weight": 0.4},
    "hr": {"dir": 1, "weight": 1.25},
    "rc": {"dir": 1, "weight": 1.12},
    "ops": {"dir": 1, "weight": 1.015},
    "xwoba": {"dir": 1, "weight": 1.15},
    "iso": {"dir": 1, "weight": 1.1},
}
dfpa_score_cols = {
    "ip": {"dir": 1, "weight": 2.0},
    "hr_per9": {"dir": -1, "weight": 0.9},
    "k_per9": {"dir": 1, "weight": 1.0},
    "whip": {"dir": -1, "weight": 1.5},
    "xfip": {"dir": -1, "weight": 1.0},
    "siera": {"dir": -1, "weight": 1.15},
}
dfb_weight = 1.0
dfba_weight = 0.85
dfp_weight = 1.0
dfpa_weight = 0.82

# get all of the score columns for maximum z-scoring
dfb_score_col_names = [x + "_score" for x in dfb_score_cols.keys()]
dfba_score_col_names = [x + "_score" for x in dfba_score_cols.keys()]
dfp_score_col_names = [x + "_score" for x in dfp_score_cols.keys()]
dfpa_score_col_names = [x + "_score" for x in dfpa_score_cols.keys()]

# filter out unqualifieds
dfba = dfba[min_pa_act]
dfpa = dfpa[min_ip_act]

### Replace Player Names with Canonical ESPN Names

Multiple inputs have different spellings or variations on player names. The ESPN ones are considered gospel for this analysis.

In [7]:
# source name -> espn name
name_replacements = {
    "Adolis García": "Adolis Garcia",
    "Alex Colomé": "Alex Colome",
    "Andrés Giménez": "Andres Gimenez",
    "Andy Ibáñez": "Andy Ibanez",
    "Avisaíl García": "Avisail Garcia",
    "Carlos Hernández": "Carlos Hernandez",
    "Carlos Rodón": "Carlos Rodon",
    "César Hernández": "Cesar Hernandez",
    "Christian Vázquez": "Christian Vazquez",
    "Domingo Germán": "Domingo German",
    "Eduardo Rodríguez": "Eduardo Rodriguez",
    "Edwin Díaz": "Edwin Diaz",
    "Elias Díaz": "Elias Diaz",
    "Eloy Jiménez": "Eloy Jimenez",
    "Enrique Hernández": "Enrique Hernandez",
    "Eugenio Suárez": "Eugenio Suarez",
    "Gary Sánchez": "Gary Sanchez",
    "Génesis Cabrera": "Genesis Cabrera",
    "Germán Márquez": "German Marquez",
    "Héctor Neris": "Hector Neris",
    "Jarlín García": "Jarlin Garcia",
    "Javier Báez": "Javier Baez",
    "Jeremy Peña": "Jeremy Pena",
    "Jesús Aguilar": "Jesus Aguilar",
    "Jesús Luzardo": "Jesus Luzardo",
    "Jesús Sánchez": "Jesus Sanchez",
    "Jonathan Loáisiga": "Jonathan Loaisiga",
    "José Abreu": "Jose Abreu",
    "José Berríos": "Jose Berrios",
    "José Cisnero": "Jose Cisnero",
    "José Iglesias": "Jose Iglesias",
    "José Ramírez": "Jose Ramirez",
    "José Ruiz": "Jose Ruiz",
    "José Suarez": "Jose Suarez",
    "José Urquidy": "Jose Urquidy",
    "Julio Rodríguez": "Julio Rodriguez",
    "Julio Urías": "Julio Urias",
    "Luis Patiño": "Luis Patino",
    "Luis Urías": "Luis Urias",
    "Manny Piña": "Manny Pina",
    "Michael Conforto": "Michael Conforto",
    "Miguel Sanó": "Miguel Sano",
    "Noé Ramirez": "Noe Ramirez",
    "Odúbel Herrera": "Odubel Herrera",
    "Omar Narváez": "Omar Narvaez",
    "Pablo López": "Pablo Lopez",
    "Ramón Laureano": "Ramon Laureano",
    "Ramón Urías": "Ramon Urias",
    "Ranger Suárez": "Ranger Suarez",
    "Reynaldo López": "Reynaldo Lopez",
    "Robinson Canó": "Robinson Cano",
    "Ronald Acuña Jr.": "Ronald Acuna Jr.",
    "Sixto Sánchez": "Sixto Sanchez",
    "Teoscar Hernández": "Teoscar Hernandez",
    "Vidal Bruján": "Vidal Brujan",
    "Yandy Díaz": "Yandy Diaz",
    "Yimi García": "Yimi Garcia",
    "Yoán Moncada": "Yoan Moncada",
    "Yusmeiro Petit": "Yusmeiro Petit",
}


def replace_names(name):
    """
    If a name is in the name_replacements list, get it. Otherwise return the rows name.
    """
    return name_replacements.get(name, name)


# replace all names in all dataframes
for df in [dfb, dfp, dfba, dfpa]:
    df["name"] = df["name"].apply(lambda x: replace_names(x))

## Calculate Z-Scores

For each of the players, we calculate a z-score based on how they compare to the rest of the league. There are several columns that we want to do this for.

- For each z-score, there is an arbitrary minimum set to inflate the league average away from non-full-time players.
- After each is totaled, the z-scores are totaled and then z-scored again to make a single number. This is done so that we can compare pitchers and batters into the same chart.

In [8]:
# only mean and stdev are filtered by minimums, so that way the player's actual
# projections are still counted normally.

# batter scores
for col in dfb_score_cols.keys():
    col_score = col + "_score"
    dfb[col_score] = (
        (dfb[col] - dfb[col][min_pa].mean()) / dfb[col][min_pa].std(ddof=0)
        * dfb_score_cols[col]["dir"]
        * dfb_score_cols[col]["weight"]
    ).round(3)

# batter actuals
for col in dfba_score_cols.keys():
    col_score = col + "_score"
    dfba[col_score] = (
        (dfba[col] - dfba[col][min_pa_act].mean()) / dfba[col][min_pa_act].std(ddof=0)
        * dfba_score_cols[col]["dir"]
        * dfba_score_cols[col]["weight"]
    ).round(3)
    
# pitcher scores
for col in dfp_score_cols.keys():
    col_score = col + "_score"
    dfp[col_score] = (
        (dfp[col] - dfp[col][min_ip].mean()) / dfp[col][min_ip].std(ddof=0)
        * dfp_score_cols[col]["dir"]
        * dfp_score_cols[col]["weight"]
    ).round(3)

# pitcher actuals
for col in dfpa_score_cols.keys():
    col_score = col + "_score"
    dfpa[col_score] = (
        (dfpa[col] - dfpa[col][min_ip_act].mean()) / dfpa[col][min_ip_act].std(ddof=0)
        * dfpa_score_cols[col]["dir"]
        * dfpa_score_cols[col]["weight"]
    ).round(3)

In [9]:
# sum all of the values into 'total_score'
dfb['total_score'] = dfb[dfb_score_col_names].sum(axis=1)
dfb['total_z_score'] = (
    dfb['total_score'] - dfb['total_score'][min_pa].mean()) / dfb['total_score'][min_pa].std(ddof=0)
dfb['total_z_score_rank'] = dfb['total_z_score'].rank(ascending=False)

dfba['total_score'] = dfba[dfba_score_col_names].sum(axis=1)
dfba['total_z_score'] = (
    dfba['total_score'] - dfba['total_score'][min_pa_act].mean()) / dfba['total_score'][min_pa_act].std(ddof=0)
dfba['total_z_score_rank'] = dfba['total_z_score'].rank(ascending=False)

dfp['total_score'] = dfp[dfp_score_col_names].sum(axis=1)
dfp['total_z_score'] = (
    dfp['total_score'] - dfp['total_score'][min_ip].mean()) / dfp['total_score'][min_ip].std(ddof=0)
dfp['total_z_score_rank'] = dfp['total_z_score'].rank(ascending=False)

dfpa['total_score'] = dfpa[dfpa_score_col_names].sum(axis=1)
dfpa['total_z_score'] = (
    dfpa['total_score'] - dfpa['total_score'][min_ip_act].mean()) / dfpa['total_score'][min_ip_act].std(ddof=0)
dfpa['total_z_score_rank'] = dfpa['total_z_score'].rank(ascending=False)

# sort by score descending
dfb.sort_values(by='total_z_score_rank', inplace=True)
dfba.sort_values(by='total_z_score_rank', inplace=True)
dfp.sort_values(by='total_z_score_rank', inplace=True)
dfpa.sort_values(by='total_z_score', ascending=False, inplace=True)


### Inflate League Mean

For the majority of the z-scores that we're taking, the actual values of them are affected by the long-tail of players who don't get much playing time. To offset this, the mean is artifically placed at halfway through our number of drafted players to account for the fact that will be replacement level for our league.

In [10]:
# decrement scores by the mean of all drafted positions
NUM_TEAMS = 8
NUM_BATTERS = 15
NUM_PITCHERS = 10

MIDDLE_BATTER_INDEX = int((NUM_TEAMS * NUM_BATTERS) / 2)
MIDDLE_PITCHER_INDEX = int((NUM_TEAMS * NUM_PITCHERS) / 2)

middle_batter_score = dfb[dfb['total_z_score_rank'] == MIDDLE_BATTER_INDEX]['total_z_score']
dfb['total_z_score'] = dfb['total_z_score'] - float(middle_batter_score)
dfb.reset_index(drop=True)

middle_pitcher_score = dfp[dfp['total_z_score_rank'] == MIDDLE_PITCHER_INDEX]['total_z_score']
dfp['total_z_score'] = dfp['total_z_score'] - float(middle_pitcher_score)
dfp.reset_index(drop=True)
print()

## Write to the Database

Prior to writing the excel files, write back to the database for safekeeping.

In [11]:
dfb.to_sql("batters_scores_ros", conn, schema="fantasy", if_exists="replace")
result = conn.execute("grant select on fantasy.batters_scores_ros to public")

dfba.to_sql("batters_scores_actuals", conn, schema="fantasy", if_exists="replace")
result = conn.execute("grant select on fantasy.batters_scores_actuals to public")

dfp.to_sql("pitchers_scores_ros", conn, schema="fantasy", if_exists="replace")
result = conn.execute("grant select on fantasy.pitchers_scores_ros to public")

dfpa.to_sql("pitchers_scores_actuals", conn, schema="fantasy", if_exists="replace")
result = conn.execute("grant select on fantasy.pitchers_scores_actuals to public")


## Create Draft Sheet

This sheet includes a number of important pieces of information for drafting specifically, so that players can look up by eligibility as well as important stats.

- Name
- Eligibility
- Positions
- Combined scores

In [12]:
# pitchers query

draft_sheet_query_p = """
select
    p.fullname
    , t."name" as teamname
    , case when p.onteamid = 0 then 0 else 1 end as owned
    , p.eligibility as elig
    , p.injurystatus as inj
    , p.percentowned as pctOwn
    , p.percentstarted as pctSt
    , p.percentChange as pctChg
    , trim(leading 'T' from pp.tier)::integer as tier
    , sc_ros.fip 
    , sc_ros.qs 
    , sc_ros.era 
    , sc_ros.whip 
    , sc_ros."k-9" 
    , sc_ros.ip 
    , sc_ros.total_z_score as score_ros
    , sc.total_z_score as score_act
    , sco.total_z_score as score_og
from fantasy.players p
    left join fantasy.teams t
        on p.onteamid = t.espn_id
    left join fantasy.pitchers_scores_actuals sc
        on p.fullname = sc."name"
    left join fantasy.pitchers_scores sco
        on p.fullname = sco."name"
    left join fantasy.pitchers_scores_ros sc_ros
        on p.fullname = sc_ros."name"
    left join fantasy.pitchers_pitcherlist_100 pp 
        on p.fullname = pp."name"
where
        p.eligibility not like '%UTIL%'
    and p.percentowned > 0
"""

# batters query

draft_sheet_query_b = """
select
    p.fullname as name
    , t."name" as teamname
    , case when p.onteamid = 0 then 0 else 1 end as owned
    , p.eligibility as elig
    , p.injurystatus as inj
    , p.percentowned as pctOwn
    , p.percentstarted as pctSt
    , p.percentChange as pctChg
    , sc_ros.rc 
    , sc_ros.obp 
    , sc_ros.slg 
    , sc_ros.hr 
    , sc_ros.k_pct 
    , sc_ros.ab 
    , sc.total_z_score as score_act
    , sco.total_z_score as score_og
    , sc_ros.total_z_score as score_ros
from fantasy.players p
    left join fantasy.teams t
        on p.onteamid = t.espn_id
    left join fantasy.batters_scores_actuals sc
        on p.fullname = sc."name"
    left join fantasy.batters_scores sco
        on p.fullname = sco."name"
    left join fantasy.batters_scores_ros sc_ros
        on p.fullname = sc_ros."name"
where
        p.eligibility like '%UTIL%'
    and p.percentowned > 0
"""
dfp = pd.read_sql(sqlalchemy.text(draft_sheet_query_p), conn)
dfb = pd.read_sql(sqlalchemy.text(draft_sheet_query_b), conn)

In [13]:
# calculate rankings and diffs by dataframe
for df in [dfp, dfb]:
    df.sort_values(by='score_ros', inplace=True, ascending=False)
    df['rank'] = df['score_ros'].rank(ascending=False, method='first', na_option='bottom')
    df.reset_index(drop=True)
    df['score_vs_act'] = df['score_act'] - df['score_ros']
    df['score_vs_og'] = df['score_ros'] - df['score_og']
print()

## Style Output

In order to make the outputs more usable, apply a number of color stylings and ranking scale sliders to the actual xlsx file.

In [15]:
dfp.head(20)

,fullname,teamname,owned,elig,inj,pctown,pctst,pctchg,tier,fip,qs,era,whip,k-9,ip,score_ros,score_act,score_og,rank,score_vs_act,score_vs_og
0,Gerrit Cole,Cole-d Crispy Bois,1,P|SP,ACTIVE,99.86,90.19,0.00,1.00,2.92,16.79,3.03,1.04,12.00,155.00,2.28,1.42,1.87,1.00,-0.86,0.41
1,Corbin Burnes,How'd it get Burnes?,1,P|SP,ACTIVE,99.88,90.55,-0.01,1.00,2.83,15.12,2.97,1.06,11.68,139.00,1.88,2.25,1.47,2.00,0.37,0.41
2,Max Scherzer,Bingo Night,1,P|SP,ACTIVE,99.88,91.12,-0.01,1.00,3.19,15.44,3.20,1.04,11.79,147.00,1.79,1.96,1.47,3.00,0.16,0.32
3,Carlos Rodon,deGrom Nom Nom,1,P|SP,ACTIVE,99.24,91.75,0.18,NaN,2.88,14.22,3.08,1.11,11.97,130.00,1.62,2.18,0.79,4.00,0.56,0.83
4,Kevin Gausman,The New Jesse Winker Record,1,P|SP,ACTIVE,99.50,88.67,0.00,2.00,3.30,15.68,3.39,1.10,9.81,151.00,1.34,2.37,0.55,5.00,1.03,0.80
5,Shane Bieber,Sho Time,1,P|SP,ACTIVE,99.73,90.65,-0.12,5.00,3.33,15.18,3.47,1.16,10.43,149.00,1.23,0.88,1.30,6.00,-0.35,-0.07
6,Brandon Woodruff,2019 All Star Team,1,P|SP,ACTIVE,98.81,92.65,-0.34,2.00,3.31,14.59,3.39,1.12,10.41,141.00,1.21,0.80,0.98,7.00,-0.41,0.23
7,Justin Verlander,Bingo Night,1,P|SP,ACTIVE,98.40,91.68,0.78,2.00,3.59,14.69,3.37,1.02,10.64,144.00,1.19,1.62,0.89,8.00,0.43,0.31
8,Robbie Ray,2019 All Star Team,1,P|SP,ACTIVE,99.70,92.92,0.01,4.00,3.79,15.19,3.67,1.16,11.05,153.00,1.17,1.02,1.17,9.00,-0.15,-0.00
9,Aaron Nola,Sho Time,1,P|SP,ACTIVE,99.04,92.75,-0.10,4.00,3.54,15.15,3.59,1.14,10.27,150.00,1.13,1.70,0.77,10.00,0.57,0.37


## Draft Sheet Preview

This is a quick look at how the players are going to appear in the final draft sheet. Useful for comparing overall pitcher v batter weights (i.e. verify that nobody should be ranked higher than trout).

In [18]:
dfb[
    (dfb['pctown'] > 5.0)
    & ((dfb['owned'] == 0 ) | (dfb['teamname'] == "2019 All Star Team"))
].sort_values(by='score_ros', ascending=False).head(150)


,name,teamname,owned,elig,inj,pctown,pctst,pctchg,rc,obp,slg,hr,k_pct,ab,score_act,score_og,score_ros,rank,score_vs_act,score_vs_og
2,Mike Trout,2019 All Star Team,1,OF|UTIL,ACTIVE,99.90,96.61,0.01,90.77,0.41,0.56,29.00,0.29,392.00,3.03,1.31,1.83,3.00,1.20,0.52
16,Nolan Arenado,2019 All Star Team,1,3B|1B/3B|UTIL,ACTIVE,99.86,98.54,0.01,76.28,0.33,0.48,25.00,0.17,483.00,1.87,0.33,0.81,17.00,1.05,0.49
22,Trea Turner,2019 All Star Team,1,2B|SS|2B/SS|UTIL,ACTIVE,99.87,98.49,0.01,79.14,0.34,0.47,19.00,0.19,486.00,0.60,0.58,0.66,23.00,-0.07,0.08
24,Josh Bell,2019 All Star Team,1,1B|1B/3B|UTIL,ACTIVE,98.62,93.24,0.30,71.87,0.36,0.48,21.00,0.21,420.00,1.55,0.42,0.60,25.00,0.95,0.18
26,George Springer,2019 All Star Team,1,OF|UTIL,ACTIVE,99.60,77.99,-0.02,70.00,0.34,0.49,24.00,0.24,416.00,1.60,0.48,0.59,27.00,1.01,0.11
32,Paul Goldschmidt,2019 All Star Team,1,1B|1B/3B|UTIL,ACTIVE,98.63,94.80,-0.01,75.66,0.35,0.46,20.00,0.24,468.00,1.02,0.25,0.42,33.00,0.59,0.17
39,J.D. Martinez,2019 All Star Team,1,OF|UTIL,ACTIVE,98.37,93.11,-0.12,73.62,0.34,0.48,21.00,0.26,453.00,1.34,0.21,0.36,40.00,0.99,0.15
42,Christian Yelich,2019 All Star Team,1,OF|UTIL,ACTIVE,97.82,95.22,0.82,71.16,0.36,0.46,20.00,0.28,422.00,1.47,0.00,0.28,43.00,1.18,0.28
60,Joey Gallo,None,0,OF|UTIL,ACTIVE,60.74,48.87,-2.51,66.35,0.34,0.47,32.00,0.44,420.00,0.34,-0.05,0.00,61.00,0.34,0.05
61,Marcell Ozuna,2019 All Star Team,1,OF|UTIL,ACTIVE,88.52,75.63,-3.40,65.13,0.32,0.44,22.00,0.23,458.00,0.81,-0.13,-0.03,62.00,0.84,0.10


# Visualizations

Here are a number of helpful visualizations to sort out the information from the draft sheet.

### ESPN Average Draft Pos vs. this Draft Sheet Ranking

Here's how players stack up against how they're being drafted on ESPN right now

In [ ]:
sb.lmplot(x="score", y="espn_adp", hue="onteamid", fit_reg=False, data=dfd[
    (dfd['percentowned'] > 5.0) &
#     ((dfd['claimed'] == 0) | (dfd['onteamid'] == 5)) &
    (dfd['catg'] == 'P') &
    (dfd['injurystatus'] == 'ACTIVE')
].sort_values(by='score', ascending=False).head(150))


In [ ]:
sb.scatterplot(x="score", y="score_ros", hue="score_diff", size="score_diff",
          data=dfd[(dfd['percentowned'] > 20.0) & (dfd['claimed'] == 0) & (dfd['catg'] == 'B')])

Here are the biggest individual differences in ranking for the top 100 players by rank in this system. If the number is negative that means this system thinks that they're better than their espn adp.

In [ ]:
dfd[(dfd['percentowned'] > 20.0) & ((dfd['claimed'] == 0) | (dfd['onteamid'] == 5)) & (dfd['pos'] == 'RP')
   ].sort_values(by='score', ascending=False).head(100)

### Team Comparisons

The individual positions and their associated depth for this year.

In [ ]:
sb.barplot(x="catg", y="score", hue="onteamid", ci=None, data=dfd[dfd['onteamid'] > 0])